In [21]:
from sim import simulation as sim
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import dask.array as da

In [22]:
point = sim.SystemState([8 for _ in np.arange(0,32)], 0)
point.perturbate()
point

coordinates: [7.96915195 8.01221088 7.99377277 8.02853586 8.02799758 7.97725926
 7.97764643 8.03018722 8.04581394 8.03759326 7.98578173 8.00009951
 8.01834629 8.0212702  7.98702508 8.00611962 8.00030832 7.95137684
 8.02728266 8.03826412 7.9864886  8.01153962 7.95753812 7.9868824
 8.04331401 8.01513781 7.98972026 8.02887301 7.98168361 8.00680987
 8.03691274 7.99361734]
time: 0

In [23]:
simulator = sim.Simulator(system_state=point, forcing=8)

In [24]:
runner = sim.SimulationRunner(simulator, integration_time = 1, chunk_length=100, out_file='/home/cucchi/phd/prova.nc')

In [25]:
runner.run()

In [26]:
data = xr.open_dataarray('/home/cucchi/phd/prova.nc', chunks={'time': 500})

In [27]:
data

<xarray.DataArray 'var' (time: 100, node: 32)>
dask.array<shape=(100, 32), dtype=float32, chunksize=(100, 32)>
Coordinates:
  * time     (time) float64 0.0 0.01 0.02 0.03 0.04 ... 0.95 0.96 0.97 0.98 0.99
  * node     (node) int32 0 1 2 3 4 5 6 7 8 9 ... 22 23 24 25 26 27 28 29 30 31

This works

In [48]:
xr.apply_ufunc(np.percentile, data, dask='allowed', input_core_dims=[['time']], vectorize=True, kwargs={'q':95})

<xarray.DataArray 'var' (node: 32)>
array([14.254335, 11.498712,  9.559041,  8.796019, 10.503936, 13.95398 ,
       11.817201, 10.448256,  9.46945 ,  8.774399, 12.446248, 13.265309,
       11.529604,  9.876963,  8.827155, 10.007007, 14.742959, 11.882041,
        9.487473,  8.53127 , 12.224711, 14.835166, 10.401242,  9.297627,
        9.265089, 12.565587, 12.02324 , 12.031843, 11.348159,  9.785212,
        8.750388, 10.712037])
Coordinates:
  * node     (node) int32 0 1 2 3 4 5 6 7 8 9 ... 22 23 24 25 26 27 28 29 30 31

This works, but does it load data before?

In [49]:
data.reduce(np.percentile, dim='time', q=95)

<xarray.DataArray 'var' (node: 32)>
array([14.254335, 11.498712,  9.559041,  8.796019, 10.503936, 13.95398 ,
       11.817201, 10.448256,  9.46945 ,  8.774399, 12.446248, 13.265309,
       11.529604,  9.876963,  8.827155, 10.007007, 14.742959, 11.882041,
        9.487473,  8.53127 , 12.224711, 14.835166, 10.401242,  9.297627,
        9.265089, 12.565587, 12.02324 , 12.031843, 11.348159,  9.785212,
        8.750388, 10.712037])
Coordinates:
  * node     (node) int32 0 1 2 3 4 5 6 7 8 9 ... 22 23 24 25 26 27 28 29 30 31

This works, but maybe is not needed (I can load time-series from just 1 noded without memory issues).

In [17]:
data1d = data.sel(node=0)
data1d = data1d.drop('node')

In [18]:
data1d.data

dask.array<getitem, shape=(100,), dtype=float32, chunksize=(100,)>

In [20]:
da.percentile(data1d.data, 95).compute()

array([14.25433464])

In [ ]:
data1d.plot()